# Process HGHI data

This notebook reads in the HGHI datasets and converts them to GeoJSON format.

Data from https://globalepidemics.org/2020/03/17/caring-for-covid-19-patients/.

See description in `data/README.md` for details.


In [ ]:
import pandas as pd
import geopandas as gpd

from covidcaremap.constants import state_name_to_abbreviation
from covidcaremap.data import external_data_path, processed_data_path

In [ ]:
hghi_state_df = pd.read_csv(external_data_path('HGHI - Hospital Capacity by State.csv'))
hghi_hrr_60_df = pd.read_csv(external_data_path('HGHI - HRR Scorecard - 60% Population.csv'))
hghi_hrr_40_df = pd.read_csv(external_data_path('HGHI - HRR Scorecard - 40% Population.csv'))
hghi_hrr_20_df = pd.read_csv(external_data_path('HGHI - HRR Scorecard - 20% Population.csv'))

# Make sure numeric values are numbers.
int_columns =[ 'Total Hospital Beds',
               'Total ICU Beds',
               'Available Hospital Beds',
               'Hospital Beds Needed, Six Months',
               'Hospital Beds Needed, Twelve Months',
               'Hospital Beds Needed, Eighteen Months',
               'ICU Beds Needed, Six Months',
               'ICU Beds Needed, Twelve Months',
               'ICU Beds Needed, Eighteen Months',
               'Potentially Available Hospital Beds*',
               'Available ICU Beds',
               'Potentially Available ICU Beds*',
               'Adult Population',
               'Population 65+',
               'Projected Infected Individuals',
               'Proejcted Hospitalized Individuals',
               'Projected Hospitalized Individuals',
               'Projected Individuals Needing ICU Care']

float_columns = [ 'Hospital Bed Occupancy Rate',
                  'ICU Bed Occupancy Rate']

percent_columns = [ 'Percentage of Available Beds Needed, Six Months',
                    'Percentage of Potentially Available Beds Needed, Six Months',
                    'Percentage of Total Beds Needed, Six Months',
                    'Percentage of Available Beds Needed, Twelve Months',
                    'Percentage of Potentially Available Beds Needed, Twelve Months',
                    'Percentage of Total Beds Needed, Twelve Months',
                    'Percentage of Available Beds Needed, Eighteen Months',
                    'Percentage of Potentially Available Beds Needed, Eighteen Months',
                    'Percentage of Total Beds Needed, Eighteen Months',
                    'Percentage of Available ICU Beds Needed, Six Months',
                    'Percentage of Potentially Available ICU Beds Needed, Six Months',
                    'Percentage of Total ICU Beds Needed, Six Months',
                    'Percentage of Available ICU Beds Needed, Twelve Months',
                    'Percentage of Potentially Available ICU Beds Needed, Twelve Months',
                    'Percentage of Total ICU Beds Needed, Twelve Months',
                    'Percentage of Available ICU Beds Needed, Eighteen Months',
                    'Percentage of Potentially Available ICU Beds Needed, Eighteen Months',
                    'Percentage of Total ICU Beds Needed, Eighteen Months']

for df in  [hghi_state_df, hghi_hrr_60_df, hghi_hrr_40_df, hghi_hrr_20_df]:
    for c in int_columns:
        if c in df.columns:
            if df[c].dtype == object:
                df[c] = df[c].str.replace(',', '').astype('int')
    for c in float_columns:
        if c in df.columns:
            df[c] = df[c].astype('float')
    for c in percent_columns:
        if c in df.columns:
            df[c] = df[c].str.replace('%', '').astype('float') / 100.0

In [ ]:
state_gdf = gpd.read_file(external_data_path('us_states.geojson'), encoding='utf-8')
hrr_gdf = gpd.read_file(external_data_path('us_hrr.geojson'), encoding='utf-8')

## Generate HRR GeoJSON

### Merge HGHI scenarios

Merge the 20, 40 and 60 scenario DFs in a dataframe that does not duplicate common properties.

In [ ]:
non_common_properties = ['Projected Infected Individuals',
       'Projected Hospitalized Individuals',
       'Projected Individuals Needing ICU Care',
       'Hospital Beds Needed, Six Months',
       'Percentage of Available Beds Needed, Six Months',
       'Percentage of Potentially Available Beds Needed, Six Months',
       'Percentage of Total Beds Needed, Six Months',
       'Hospital Beds Needed, Twelve Months',
       'Percentage of Available Beds Needed, Twelve Months',
       'Percentage of Potentially Available Beds Needed, Twelve Months',
       'Percentage of Total Beds Needed, Twelve Months',
       'Hospital Beds Needed, Eighteen Months',
       'Percentage of Available Beds Needed, Eighteen Months',
       'Percentage of Potentially Available Beds Needed, Eighteen Months',
       'Percentage of Total Beds Needed, Eighteen Months',
       'ICU Beds Needed, Six Months',
       'Percentage of Available ICU Beds Needed, Six Months',
       'Percentage of Potentially Available ICU Beds Needed, Six Months',
       'Percentage of Total ICU Beds Needed, Six Months',
       'ICU Beds Needed, Twelve Months',
       'Percentage of Available ICU Beds Needed, Twelve Months',
       'Percentage of Potentially Available ICU Beds Needed, Twelve Months',
       'Percentage of Total ICU Beds Needed, Twelve Months',
       'ICU Beds Needed, Eighteen Months',
       'Percentage of Available ICU Beds Needed, Eighteen Months',
       'Percentage of Potentially Available ICU Beds Needed, Eighteen Months',
       'Percentage of Total ICU Beds Needed, Eighteen Months']

hghi_hrr_60_subset_df = hghi_hrr_60_df[['HRR'] + non_common_properties].add_suffix(' (60%)')
hghi_hrr_40_subset_df = hghi_hrr_40_df[['HRR'] + non_common_properties].add_suffix(' (40%)')
hghi_hrr_20_subset_df = hghi_hrr_20_df[['HRR'] + non_common_properties].add_suffix(' (20%)')

common_properties = [
    'HRR', 'Total Hospital Beds', 'Total ICU Beds',
    'Available Hospital Beds', 'Potentially Available Hospital Beds*',
    'Available ICU Beds', 'Potentially Available ICU Beds*',
    'Adult Population', 'Population 65+' 
]

hghi_hrr_common_subset_df = hghi_hrr_60_df[common_properties]

In [ ]:
hghi_hrr_merged_df = hghi_hrr_common_subset_df.set_index('HRR').join(
    [hghi_hrr_60_subset_df.set_index('HRR (60%)'),
     hghi_hrr_40_subset_df.set_index('HRR (40%)'),
     hghi_hrr_20_subset_df.set_index('HRR (20%)')]
).reset_index()
hghi_hrr_merged_df

### Reformat HRR name in HGHI data

HRRs are defined as `city, state` in the HGHI data, and `state- city` in the HRR geojson.

Also, there are some differing names beyond formatting. Account for this with a explicit mapping.

**NOTE:** HGHI data is missing 'FL- ST PETERSBURG' HRR that is contained in the HRR region dataset.

In [ ]:
print('HGHI data contains {} HRRs, region data contains {}'.format(
    len(hghi_hrr_merged_df['HRR']), len(hrr_gdf['HRRCITY'])))

In [ ]:
# HGHI missing 'FL- ST PETERSBURG'
hghi_to_hrr_mapping = {
    'AK- ANCHORAGE': 'AK-Anchorage',
    'CA- ALAMEDA COUNTY': 'CA- ALAMEDA CO.',
    'CA- CONTRA COSTA COUNTY': 'CA- CONTRA COSTA CO.',
    'CA- ORANGE COUNTY': 'CA- ORANGE CO.',
    'CA- PALM SPRINGS/RANCHO MIRA': 'CA- PALM SPR/RANCHO MIR.',
    'CA- SAN MATEO COUNTY': 'CA- SAN MATEO CO.',
    'FL- ST. PETERSBURG': 'FL- ST PETERSBURG',
    'HI- HONOLULU': 'HI-Honolulu',
    'MI- ST. JOSEPH': 'MI- ST JOSEPH',
    'MN- ST. CLOUD': 'MN- ST CLOUD',
    'MN- ST. PAUL': 'MN- ST PAUL',
    'MO- ST. LOUIS': 'MO- ST LOUIS',
    'ND- FARGO/MOORHEAD MN': 'ND- FARGO MOORHEAD -MN',
    'NY- MANHATTAN': 'NY- NEW YORK'
}

In [ ]:
def reformat_hrr_name(hrr):
    city, state = hrr.split(', ')
    name = '{}- {}'.format(state, city.upper())
    if name in hghi_to_hrr_mapping:
        return hghi_to_hrr_mapping[name]
    return name

hghi_hrr_merged_df['HRRCITY'] = hghi_hrr_merged_df['HRR'].apply(reformat_hrr_name)

In [ ]:
hghi_hrr_gdf = gpd.GeoDataFrame(
    hghi_hrr_merged_df.set_index('HRRCITY').join(hrr_gdf.set_index('HRRCITY')).reset_index()
, crs=4326)

In [ ]:
hghi_hrr_gdf.to_file(processed_data_path('hghi_hrr_data.geojson'), 
                  encoding='utf-8', 
                  driver='GeoJSON')

## Generate State GeoJSON

Note: there is not a differentiation between scenarios. From `Projected Infected Individuals` / `Adult Population` it seems to be a 60% scenario.

In [ ]:
abbrv_to_name = dict([(v, k) for k, v in state_name_to_abbreviation.items()])
hghi_state_df['State Name'] = hghi_state_df['State'].apply(lambda x: abbrv_to_name[x])

In [ ]:
hghi_state_gdf = hghi_state_df.set_index('State Name').join(
        state_gdf.drop(columns=['STATE']).set_index('NAME')
).reset_index()

hghi_state_gdf =gpd.GeoDataFrame(hghi_state_gdf, crs=4326)

In [ ]:
hghi_state_gdf.to_file(processed_data_path('hghi_state_data.geojson'), 
                  encoding='utf-8', 
                  driver='GeoJSON')